In [1]:
import tensorflow as tf
import util

x = tf.convert_to_tensor([[[1.,2.,3.,4.,5.,6.,7.,8.],[5.,6.,7.,8.,9.,10.,11.,12.],[9.,10.,11.,12.,13.,14.,15.,16.],[13.,14.,15.,16.,17,18.,19.,20.],[17.,18.,19.,20.,21.,22.,23.,24.],[21.,22.,23.,24.,25.,26,27.,28.]]])
print(x)
#,[17,18,19,20],[21,22,23,24],[25,26,27,28]]])

tf.Tensor(
[[[ 1.  2.  3.  4.  5.  6.  7.  8.]
  [ 5.  6.  7.  8.  9. 10. 11. 12.]
  [ 9. 10. 11. 12. 13. 14. 15. 16.]
  [13. 14. 15. 16. 17. 18. 19. 20.]
  [17. 18. 19. 20. 21. 22. 23. 24.]
  [21. 22. 23. 24. 25. 26. 27. 28.]]], shape=(1, 6, 8), dtype=float32)


In [2]:
import transforms as transf
xf = transf.fft2d_inner((6,8),dtype='float64')(tf.reshape(x,(1,6,8,1,1)))

In [3]:
xf.shape

TensorShape([1, 6, 5, 1, 1])

In [4]:
x_downsample = transf.ifft2d_inner((3,4))(util.freq_downsample(xf))
x_shift_downsample = transf.ifft2d_inner((3,4))(util.freq_shift_downsample(xf))
x_col_downsample = transf.ifft2d_inner((6,4))(util.freq_col_downsample(xf))
x_col_shift_downsample = transf.ifft2d_inner((6,4))(util.freq_col_shift_downsample(xf))

In [5]:
#print(tf.squeeze(x_downsample,axis=[-2,-1]))
#print(tf.strided_slice(x,begin=[0,0,0],end =[0,6,8],strides=[1,2,2],end_mask=1))


print(tf.reduce_max(tf.abs(tf.squeeze(x_downsample,axis=[-2,-1]) - tf.strided_slice(x,begin=[0,0,0],end =[0,6,8],strides=[1,2,2],end_mask=1))))

tf.Tensor(1.4305115e-06, shape=(), dtype=float32)


In [6]:
print(tf.squeeze(x_shift_downsample,axis=[-2,-1]))
print(tf.reduce_max(tf.abs(tf.squeeze(x_shift_downsample,axis=[-2,-1]) - tf.strided_slice(x,begin=[0,1,0],end =[0,6,8],strides=[1,2,2],end_mask=1))))

tf.Tensor(
[[[ 5.000005  7.000002  9.000001 11.000004]
  [12.999999 14.999998 16.999998 18.999998]
  [20.999998 22.999996 24.999998 27.      ]]], shape=(1, 3, 4), dtype=float32)
tf.Tensor(4.7683716e-06, shape=(), dtype=float32)


In [7]:
print(tf.squeeze(x_col_downsample,axis=[-2,-1]))
print(tf.reduce_max(tf.abs(tf.squeeze(x_col_downsample,axis=[-2,-1]) - tf.strided_slice(x,begin=[0,0,0],end =[0,6,8],strides=[1,1,2],end_mask=1))))

tf.Tensor(
[[[ 1.0000014  3.0000014  5.0000014  7.0000014]
  [ 5.0000052  7.0000033  9.000002  11.000004 ]
  [ 9.000003  11.000001  13.000001  15.000003 ]
  [12.999998  14.999998  16.999998  18.999998 ]
  [17.        19.        21.        23.       ]
  [20.999996  22.999994  24.999996  26.999998 ]]], shape=(1, 6, 4), dtype=float32)
tf.Tensor(5.722046e-06, shape=(), dtype=float32)


In [8]:
print(tf.squeeze(x_col_shift_downsample,axis = [-2,-1]))
print(tf.reduce_max(tf.abs(tf.squeeze(x_col_shift_downsample,axis=[-2,-1]) - tf.strided_slice(x,begin=[0,0,1],end =[0,6,8],strides=[1,1,2],end_mask=1))))

tf.Tensor(
[[[ 2.0000017  4.0000033  6.000002   8.000002 ]
  [ 6.000003   8.000007  10.000003  12.000003 ]
  [10.        12.000001  13.999998  16.       ]
  [13.999998  15.999998  18.        19.999998 ]
  [18.        19.999996  22.        23.999996 ]
  [21.999998  23.999996  25.999994  27.999996 ]]], shape=(1, 6, 4), dtype=float32)
tf.Tensor(6.67572e-06, shape=(), dtype=float32)


In [9]:
shift_concat = util.AlternateConcat(shape=x_downsample.shape[1:],num_of_items=2,axis=1,dtype = tf.as_dtype('float32'))
cols_concat = util.AlternateConcat(x_col_downsample.shape[1:],num_of_items=2,axis=2,dtype = tf.as_dtype('float32'))

In [10]:
col_downsample_recon = shift_concat((x_downsample,x_shift_downsample))
print(tf.squeeze(col_downsample_recon,axis=[-1,-2]))
x_recon = cols_concat((col_downsample_recon,x_col_shift_downsample))
print(tf.squeeze(x_recon,axis=[-1,-2]))
print(tf.reduce_max(tf.abs(tf.squeeze(x_recon,axis=[-1,-2]) - x)))

tf.Tensor(
[[[ 1.0000012  3.0000005  5.000001   7.0000014]
  [ 5.000005   7.000002   9.000001  11.000004 ]
  [ 9.000001  10.999999  12.999999  15.000001 ]
  [12.999999  14.999998  16.999998  18.999998 ]
  [17.        19.        21.        23.       ]
  [20.999998  22.999996  24.999998  27.       ]]], shape=(1, 6, 4), dtype=float32)
tf.Tensor(
[[[ 1.0000012  2.0000017  3.0000005  4.0000033  5.000001   6.000002
    7.0000014  8.000002 ]
  [ 5.000005   6.000003   7.000002   8.000007   9.000001  10.000003
   11.000004  12.000003 ]
  [ 9.000001  10.        10.999999  12.000001  12.999999  13.999998
   15.000001  16.       ]
  [12.999999  13.999998  14.999998  15.999998  16.999998  18.
   18.999998  19.999998 ]
  [17.        18.        19.        19.999996  21.        22.
   23.        23.999996 ]
  [20.999998  21.999998  22.999996  23.999996  24.999998  25.999994
   27.        27.999996 ]]], shape=(1, 6, 8), dtype=float32)
tf.Tensor(6.67572e-06, shape=(), dtype=float32)


In [9]:
x2 = tf.random.normal((8,66,66,32,1))

In [10]:
xf2 = transf.fft2d_inner((66,66),dtype='float64')(x2)

In [11]:
x_col_downsample_freq = util.freq_col_downsample(xf2)

In [12]:
x_col_downsample_freq.shape

TensorShape([8, 66, 17, 32, 1])

In [13]:
x_col_downsample = transf.ifft2d_inner((66,33))(x_col_downsample_freq)

In [14]:
x_col_downsample.shape

TensorShape([8, 66, 33, 32, 1])